In [56]:
from lxml import html
from lxml import objectify
import requests
import json
import pandas as pd
import csv
import numpy as np
import Utils
from importlib import reload

In [61]:
class Scraper_BackCountry:
    def __init__(self):
        self.BASE_URL = "https://www.backcountry.com"
        self.HEADERS = {
            'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36",
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'en-US,en;q=0.9',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'referer': 'https://www.google.com/',
        }
    
    def getSpecs(self, url):
        tree = self.get(url)

        specNames = tree.xpath('//dt[@class="prod-details-accordion__techspec-name js-techspec-name"]/text()')
        specValues = tree.xpath('//dd[@class="prod-details-accordion__techspec-value js-techspec-value"]/text()')

        img = tree.xpath('//img[@class="ui-flexslider__img js-flexslider-img qa-flexslider__img"]')[0]
        img = html.tostring(img, encoding="unicode")
        img = img.split('data-src="//')[1].split('"')[0]
        name = tree.xpath('//title/text()')[0].split(" |")[0]
        summary = tree.xpath('//script[@type="text/javascript"]/text()')[3]
        summary = summary.split('BC.product = ')[1].split(';BC.product')[0]
        summary = json.loads(summary)
        currency = tree.xpath('//meta[@itemprop="priceCurrency"]/@content')[0]
        avgRating, numRatings = self.calculateRating(tree.xpath('//div[@class="reviews-wrap"]')[0])

        data = {}
        data["URL"] = url
        data["Name"] = summary["displayName"]
        data["Brand"] = summary["brand"]["displayName"]
        data["lowPrice"] = summary["lowListPrice"].split("$")[1]  #maybe add symbols for different currencies
        data["highPrice"] = summary["highListPrice"].split("$")[1]
        data["PriceCurrency"] = currency
        data["Image"] = img
        data['Rating'] = avgRating
        data['numRatings'] = numRatings
        for i, item in enumerate(specValues):
            data[specNames[i]] = item
        return data

    
    def getPages(self, url):
        tree = self.get(url)
        numPagesList = tree.xpath('//li[@class="page-link page-number qa-page-link"]/.//a/text()')
        if(len(numPagesList) == 0):
            return [url]

        urlList = []
        numPages = int(numPagesList[-1])
        if "?" in url:
            pagesStr1 = "&page="
        else:
            pagesStr1 = "?page="
        pagesStr2 = "&pagesize=42"    # for some reason always added when changing page manually on website
        for i in range(numPages):
            pageURL = url + pagesStr1 + str(i) + pagesStr2
            urlList.append(pageURL)
        return urlList
    
    
    
    def getProducts(self, urls):
        urlPages = self.getPages(urls) # get urls of each page of the website category
        urlProducts = []
        for url in urlPages: #get urls of each product on each page and append to list
            productsURL = self.getProductsFromPage(url)
            for productURL in productsURL:
                urlProducts.append(productURL)
        products = []
        for prodURL in urlProducts:
            print(prodURL)
            specs = self.getSpecs(self.BASE_URL + prodURL)
            products.append(specs) # append product list to specs
        return products


    def scrape(self, categories):
        products = {}
        for category in categories:
            print(category)
            products[category] = []
            for url in categories[category]:
                print(url)
                products[category].append(self.getProducts(url))
                Utils.save_json(products)
        return products
    
    
    
    # calculates produt rating from reviews listed on product page
    def calculateRating(self, tree):
        #get number of reviews of each numbered rating
        numFive = len(tree.xpath('//div[@class="rating-value-5 review-header-rating"]'))
        numFour = len(tree.xpath('//div[@class="rating-value-4 review-header-rating"]'))
        numThree = len(tree.xpath('//div[@class="rating-value-3 review-header-rating"]'))
        numTwo = len(tree.xpath('//div[@class="rating-value-2 review-header-rating"]'))
        numOne = len(tree.xpath('//div[@class="rating-value-1 review-header-rating"]'))
        
        numSum = numFive + numFour + numThree + numTwo + numOne
        # return -1 if product has no ratings
        if(numSum == 0):
            return -1, 0
        #calculate average rating and total number of ratings
        return ( ( 5 * numFive + 4 * numFour + 3 * numThree + 2 * numTwo + 1 * numOne) / numSum ), numSum
    
    def getProductsFromPage(self, url):
        tree = self.get(url)

        products = tree.xpath('//div[@aria-label="Product"]/.//a/@href')
        return products
    
        # helper function for get requests
    def get(self, url):
        return Utils.get(url, self.HEADERS)

In [ ]:
scraper = Scraper_BackCountry()
data = scraper.scrape(categories)
Utils.save_json(data)
print("done!")

Backpack
https://www.backcountry.com/technical-daypacks
/osprey-packs-talon-22-backpack-1200-1300cu-in?skid=OSP007Z-ULTBL-SM&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MToxOmJjLXRlY2huaWNhbC1kYXlwYWNrcw==
/osprey-packs-tempest-30-backpack-womens-1709-1831cu-in-osp008d?skid=OSP008D-IRIBL-XSS&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MToyOmJjLXRlY2huaWNhbC1kYXlwYWNrcw==
/osprey-packs-talon-33-backpack-1900-2000cu-in-osp007y?skid=OSP007Y-ULTBL-SM&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MTozOmJjLXRlY2huaWNhbC1kYXlwYWNrcw==
/mountain-hardwear-scrambler-35l-backpack?skid=MHW01CQ-BKMUL-SM&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MTo0OmJjLXRlY2huaWNhbC1kYXlwYWNrcw==
/ortovox-peak-45-backpack-2750cu-in?skid=ORT003H-SAFBLU-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MTo1OmJjLXRlY2huaWNhbC1kYXlwYWNrcw==
/patagonia-ascensionist-55l-backpack?skid=PATZ9HR-FIR-LXL&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MTo2OmJjLXRlY2huaWNhbC1kYXlwYWNrcw==
/mystery-ranch-scree-backpack-1953cu-in?skid=MYT002Z-CAR-SM&ti=UExQIEN

/mystery-ranch-tower-47l-daypack?skid=MYTD00Y-BLA-LXL&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MjoxODpiYy10ZWNobmljYWwtZGF5cGFja3M=
/mountain-hardwear-multi-pitch-30l-backpack?skid=MHW01CO-DARPINBK-ML&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MjoxOTpiYy10ZWNobmljYWwtZGF5cGFja3M=
/hyperlite-mountain-gear-dyneema-summit-pack-1831cu-in?skid=HMG000J-WH-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MjoyMDpiYy10ZWNobmljYWwtZGF5cGFja3M=
/mountain-hardwear-ul-20-backpack?skid=MHWZ994-DIV-REG&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MjoyMTpiYy10ZWNobmljYWwtZGF5cGFja3M=
/black-diamond-trail-blitz-12l-backpack?skid=BLD00YM-BK-ONESIZ&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MjoyMjpiYy10ZWNobmljYWwtZGF5cGFja3M=
/mountain-hardwear-multi-pitch-20l-backpack?skid=MHW01CP-BK-REG&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MjoyMzpiYy10ZWNobmljYWwtZGF5cGFja3M=
/arc-teryx-brize-32-backpack-1950cu-in?skid=ARC00NU-YUK-REG&ti=UExQIENhdDpUZWNobmljYWwgRGF5cGFja3M6MjoyNDpiYy10ZWNobmljYWwtZGF5cGFja3M=
/osprey-packs-sirrus-24-b

In [52]:
categories = {}

backPackURLs = [
    'https://www.backcountry.com/technical-daypacks',
    'https://www.backcountry.com/hydration-packs',
    'https://www.backcountry.com/backpacking-packs',
    'https://www.backcountry.com/kid-carriers'
]

tentURLS = [
    'https://www.backcountry.com/3-season-backpacking-tents',
    'https://www.backcountry.com/4-season-tents',
    
]

bivyURLs = [
    'https://www.backcountry.com/bivys'
]

sleepingBagURLs = [
    'https://www.backcountry.com/sleeping-bags',   
]

categories['Backpack'] = backPackURLs
categories['Tent'] = tentURLS
categories['Bivy'] = bivyURLs
categories['Sleeping Bag'] = sleepingBagURLs